<a href="https://colab.research.google.com/github/BlackCat17pipo/TEST1/blob/main/Ayudantia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy pandas matplotlib

In [ ]:
"""
Barrido de tasas de aprendizaje (eta) con costo computacional
- Etas: [1e-4, 1e-3, 1e-2, 5e-2, 1e-1, 2e-1, 5e-1, 1.0]
- Pocas épocas (EPOCHS=200) para comparar convergencia
- Mide:
  * wall_time_s (tiempo total por eta)
  * ms_per_epoch (promedio por época)
  * flops_epoch_aprox y flops_total_aprox

Genera:
  * A_eta_sweep_mse_vs_epoch.png
  * A_eta_sweep_final_mse_vs_eta.png
  * ejemploA_eta_sweep_hist.csv     (historial completo)
  * ejemploA_eta_sweep_summary.csv  (resumen con costos)
"""
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# -------------------------
# 1) Datos (m2 -> UF)
# -------------------------
X = np.array([30, 45, 60, 70, 90, 110], dtype=float).reshape(-1, 1)
y = np.array([12.0, 16.0, 20.0, 23.0, 28.0, 34.0], dtype=float)
Xb = np.c_[X, np.ones(len(X))]           # n x d_b  (d_b=2: [w, b])

n = Xb.shape[0]
d_b = Xb.shape[1]

# -------------------------
# 2) Hiperparámetros
# -------------------------
ETAS = [1e-4, 1e-3, 1e-2, 5e-2, 1e-1, 2e-1, 5e-1, 1.0]
EPOCHS = 200  # pocas épocas

# Estimación muy simple de FLOPs por época (orden de magnitud):
# - y_hat = Xb @ w                      ~ 2*n*d_b
# - error = y - y_hat                   ~ n
# - grad = (-2/n) * (Xb.T @ error)      ~ 2*n*d_b + d_b  (escala)
# - w -= eta * grad                     ~ 2*d_b
# Total aprox por época: 4*n*d_b + n + 3*d_b
FLOPS_EPOCH_APROX = 4*n*d_b + n + 3*d_b

def run_gd(eta: float, epochs: int):
    """Entrena por GD; retorna historias de MSE, w, b y tiempo total."""
    w = np.zeros(d_b)  # [w, b]
    mse_hist, w_hist, b_hist = [], [], []

    t0 = time.perf_counter()
    for ep in range(epochs + 1):
        y_hat = Xb @ w
        mse = float(np.mean((y - y_hat) ** 2))
        mse_hist.append(mse); w_hist.append(float(w[0])); b_hist.append(float(w[1]))

        if ep < epochs:
            error = y - y_hat
            grad = (-2.0 / n) * (Xb.T @ error)   # gradiente MSE respecto [w, b]
            w -= eta * grad
    t1 = time.perf_counter()

    wall_time_s = t1 - t0
    return (np.array(mse_hist), np.array(w_hist), np.array(b_hist), wall_time_s)

# -------------------------
# 3) Barrido y recopilación
# -------------------------
history_rows, summ_rows = [], []
for eta in ETAS:
    mse_hist, w_hist, b_hist, wall_time_s = run_gd(eta, EPOCHS)

    # Historial completo
    for ep in range(EPOCHS + 1):
        history_rows.append({
            "eta": eta,
            "epoch": ep,
            "w": float(w_hist[ep]),
            "b": float(b_hist[ep]),
            "mse": float(mse_hist[ep]),
        })

    # Resumen por eta
    best_ep = int(np.argmin(mse_hist))
    best_mse = float(np.min(mse_hist))
    final_mse = float(mse_hist[-1])

    flops_total_aprox = FLOPS_EPOCH_APROX * (EPOCHS)  # sin contar la época 0 (solo forward)
    ms_per_epoch = (wall_time_s / EPOCHS) * 1000.0

    summ_rows.append({
        "eta": eta,
        "best_epoch": best_ep,
        "best_mse": best_mse,
        "final_mse": final_mse,
        "wall_time_s": wall_time_s,
        "ms_per_epoch": ms_per_epoch,
        "flops_epoch_aprox": FLOPS_EPOCH_APROX,
        "flops_total_aprox": flops_total_aprox
    })

history_df = pd.DataFrame(history_rows)
summary_df = pd.DataFrame(summ_rows).sort_values(by="final_mse")

# -------------------------
# 4) Guardar CSVs
# -------------------------
history_df.to_csv("ejemploA_eta_sweep_hist.csv", index=False)
summary_df.to_csv("ejemploA_eta_sweep_summary.csv", index=False)

# -------------------------
# 5) Gráfico: MSE vs época (todas las etas)
# -------------------------
plt.figure()
for eta in ETAS:
    sub = history_df[history_df["eta"] == eta]
    plt.plot(sub["epoch"].values, sub["mse"].values, label=f"eta={eta}")
plt.xlabel("Época")
plt.ylabel("MSE")
plt.title("Barrido de tasa de aprendizaje (MSE vs época)")
plt.legend()
plt.savefig("A_eta_sweep_mse_vs_epoch.png", dpi=160, bbox_inches="tight")
plt.close()

# -------------------------
# 6) Gráfico: MSE final vs eta (log-x)
# -------------------------
plt.figure()
plt.plot(summary_df["eta"].values, summary_df["final_mse"].values, marker="o")
plt.xscale("log")
plt.xlabel("eta (escala log)")
plt.ylabel("MSE final")
plt.title("MSE final vs tasa de aprendizaje")
plt.savefig("A_eta_sweep_final_mse_vs_eta.png", dpi=160, bbox_inches="tight")
plt.close()

# -------------------------
# 7) Sugerencias según criterio y costo
# -------------------------
best_by_final = summary_df.iloc[0]["eta"]
best_by_min = summary_df.loc[summary_df["best_mse"].idxmin(), "eta"]
fastest_eta = summary_df.sort_values(by="ms_per_epoch").iloc[0]["eta"]

print(f"Mejor eta por MSE FINAL (@{EPOCHS} épocas): {best_by_final}")
print(f"Mejor eta por MÍNIMO MSE alcanzado: {best_by_min}")
print(f"Eta más rápida (ms/época): {fastest_eta}")
print("\nResumen con costos guardado en: ejemploA_eta_sweep_summary.csv")



/tmp/ipython-input-571537072.py:53: RuntimeWarning: overflow encountered in square
  mse = float(np.mean((y - y_hat) ** 2))
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:127: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(arr, axis, dtype, out, keepdims, where=where)
/tmp/ipython-input-571537072.py:58: RuntimeWarning: overflow encountered in matmul
  grad = (-2.0 / n) * (Xb.T @ error)   # gradiente MSE respecto [w, b]
/tmp/ipython-input-571537072.py:59: RuntimeWarning: invalid value encountered in subtract
  w -= eta * grad


Mejor eta por MSE FINAL (@200 épocas): 0.0001
Mejor eta por MÍNIMO MSE alcanzado: 0.0001
Eta más rápida (ms/época): 1.0

Resumen con costos guardado en: ejemploA_eta_sweep_summary.csv
